In [3]:
# imports
from itertools import product, groupby, combinations, permutations
from itertools import cycle
from collections import deque
from collections.abc import Iterable
import math

In [8]:
INPUTS_DIR = "inputs"

# Common Classes

In [ ]:
class IntCodeComputer:

    def __init__(self, mem, phase=None):
        self.debug = False

        self.mem = {k:v for k,v in enumerate(mem)}
        self.pointer = 0
        self.output = []
        self.input_val = deque()
        self.relative_base = 0

        self.ops = {
            1: self._sum,
            2: self._mult,
            3: self._input,
            4: self._output,
            5: self._jump_if_true,
            6: self._jump_if_false,
            7: self._less_than,
            8: self._equals,
            9: self._adjust_relative_base,
            99: self._halt
        }

        if phase:
            self._set_phase(phase)

    def run(self, input_val):
        if isinstance(input_val, Iterable):
            self.input_val.extend(input_val)
        else:
            self.input_val.append(input_val)

        while (not self.halted):
            func, *modes = self.decode(self.mem[self.pointer])
            if func == self.ops[3] and not self.input_val:
                break

            func(*modes)

        return self.output[-1]
            
    def _set_phase(self, phase):
        try:
            self.run(phase)
        except IndexError:
            pass

    @property
    def halted(self):
        return self.mem[self.pointer] == 99

    def value(self, val, mode):
        if mode == 0:
            return self.mem.get(val, 0)
        elif mode == 1:
            return val
        elif mode == 2:
            return self.mem.get(self.relative_base + val, 0)
        else:
            print("something is wrong")

    def store_value(self, val, mode):
        if mode == 2:
            return self.relative_base + val
        else:
            return val

    def decode(self, opcode):
        instruction = opcode % 100
        opcode = opcode // 100

        mode1 = opcode % 10
        opcode = opcode // 10

        mode2 = opcode % 10
        opcode = opcode // 10

        mode3 = opcode % 10
        opcode = opcode // 10

        return self.ops.get(instruction), mode1, mode2, mode3

    # OPCODE 1
    def _sum(self, *modes):
        p = self.pointer
        param1 = self.value(self.mem[p+1], modes[0])
        param2 = self.value(self.mem[p+2], modes[1])
        store = self.store_value(self.mem[p+3], modes[2])
        self.mem[store] = param1 + param2        
        self.pointer += 4
    
    # OPCODE 2
    def _mult(self,*modes):
        p = self.pointer
        param1 = self.value(self.mem[p+1], modes[0])
        param2 = self.value(self.mem[p+2], modes[1])
        store = self.store_value(self.mem[p+3], modes[2])
        self.mem[store] = param1 * param2
        self.pointer += 4

    # OPCODE 3
    def _input(self,*modes):
        store = self.store_value(self.mem[self.pointer+1], modes[0])
        self.mem[store] = self.input_val.popleft()
        self.pointer += 2

    # OPCODE 4
    def _output(self,*modes):
        target = self.mem[self.pointer+1]
        self.output.append(self.value(target, modes[0]))
        self.pointer += 2
    
    # OPCODE 5
    def _jump_if_true(self, *modes):
        p = self.pointer
        param = self.value(self.mem[p+1], modes[0])
        if param != 0:
            self.pointer = self.value(self.mem[p+2], modes[1])
        else:
            self.pointer += 3

    # OPCODE 6
    def _jump_if_false(self, *modes):
        p = self.pointer
        param = self.value(self.mem[p+1], modes[0])
        if param == 0:
            self.pointer = self.value(self.mem[p+2], modes[1])
        else:
            self.pointer += 3

    # OPCODE 7
    def _less_than(self, *modes):
        p = self.pointer
        param1 = self.value(self.mem[p+1], modes[0])
        param2 = self.value(self.mem[p+2], modes[1])
        store = self.store_value(self.mem[p+3], modes[2])
        self.mem[store] = 1 if param1 < param2 else 0
        self.pointer += 4

    # OPCODE 8
    def _equals(self, *modes):
        p = self.pointer
        param1 = self.value(self.mem[p+1], modes[0])
        param2 = self.value(self.mem[p+2], modes[1])
        store = self.store_value(self.mem[p+3], modes[2])
        self.mem[store] = 1 if param1 == param2 else 0
        self.pointer += 4

    def _adjust_relative_base(self, *modes):
        base = self.mem[self.pointer+1]
        self.relative_base += self.value(base, modes[0])
        self.pointer += 2

    def _halt(self):
        print("halt")

# Day 1

In [9]:
def fuel(module):
    return (module // 3) - 2

sum = 0
with open(f"{INPUTS_DIR}/day01", "r") as fin:
    for line in fin.readlines():
        module = int(line.strip())
        sum += fuel(module)
sum

3167282

In [ ]:
def recurse_fuel(module):
    if fuel(module) <= 0:
        return 0
    else:
        f = fuel(module)
        return f + recurse_fuel(f)

sum = 0
with open(f"{INPUTS_DIR}/day01", "r") as fin:
    for line in fin.readlines():
        module = int(line.strip())
        sum += recurse_fuel(module)
sum

4748063

# Day 2

In [ ]:
def Intcode(ops):
    pointer = 0
    while (ops[pointer] != 99):
        store = ops[pointer+3]
        if ops[pointer] == 1:
            ops[store] = ops[ops[pointer+1]] + ops[ops[pointer+2]]
        elif ops[pointer] == 2:
            ops[store] = ops[ops[pointer+1]] * ops[ops[pointer+2]]
        else:
            print(f"something went wrong {pointer}")
        pointer += 4

def day2(ops, noun, verb):
    ops[1] = noun
    ops[2] = verb
    Intcode(ops)
    return ops[0]

with open(f"{INPUTS_DIR}/day02", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")]

day2(ops, 12, 2)

4462686

In [ ]:
with open(f"{INPUTS_DIR}/day02", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")]

for noun, verb in product(range(0,100), range(0,100)):
    _ops = ops.copy()
    if day2(_ops, noun, verb) == 19690720:
        print(100 * noun + verb)

5936


# Day 3

In [ ]:
def wire_path(wire):
    visited = []
    x,y = 0,0

    for move in wire:
        direction = move[0]
        distance = int(move[1:])

        if direction == 'R':
            path = [(x + newx,y) for newx in range(1, distance+1)]
        elif direction == 'L':
            path = [(x - newx,y) for newx in range(1, distance+1)]
        elif direction == 'U':
            path = [(x,y+newy) for newy in range(1, distance+1)]
        elif direction == 'D':
            path = [(x,y-newy) for newy in range(1, distance+1)]

        x,y = path[-1][0], path[-1][1]
        visited.extend(path)
    
    return visited

def closest_intersection(wire1, wire2):
    path1 = set(wire_path(wire1))
    path2 = set(wire_path(wire2))
    
    distances = [abs(intersect[0]) + abs(intersect[1]) for intersect in path1.intersection(path2)]
    return min(distances)

with open(f"{INPUTS_DIR}/day03", "r") as fin:
    wire1 = fin.readline().split(",")
    wire2 = fin.readline().split(",")

closest_intersection(wire1, wire2)

1264

In [ ]:
with open(f"{INPUTS_DIR}/day03", "r") as fin:
    wire1 = fin.readline().split(",")
    wire2 = fin.readline().split(",")

def min_wire_steps(wire1, wire2):
    path1 = wire_path(wire1)
    path2 = wire_path(wire2)

    intersects = set(path1).intersection(set(path2))
    steps_to_intersect = [
        path1.index(intersect) + path2.index(intersect) + 2 
        for intersect in intersects
    ]

    return min(steps_to_intersect)

min_wire_steps(wire1, wire2)

37390

# Day 4

In [ ]:
def non_decreasing_digits(digits):
    valid = True
    for i, digit in enumerate(digits[0:len(digits)-1]):
        valid &= digit <= digits[i+1]
    return valid

def has_adjacent_digits(digits):
    valid = False
    for i, digit in enumerate(digits[0:len(digits)-1]):
        valid |= digit == digits[i+1]
    return valid

passwords = [list(str(i)) for i in range(254032,789860)]
valid_passwords = filter(non_decreasing_digits, passwords)
valid_passwords = filter(has_adjacent_digits, valid_passwords)
print(len(list(valid_passwords)))


1033


In [ ]:
def has_adjacent_digits_max_two(digits):
    groups = [list(g) for _, g in groupby(digits)]
    group_lens = list(map(len, groups))
    return (2 in group_lens)

valid_passwords = filter(non_decreasing_digits, passwords)
valid_passwords = filter(has_adjacent_digits_max_two, valid_passwords)
print(len(list(valid_passwords)))


670


# Day 5

In [ ]:
with open(f"{INPUTS_DIR}/day05", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")]  
    cpu = IntCodeComputer(ops)
    out = cpu.run(1)
    print(out)

3122865


In [ ]:
with open(f"{INPUTS_DIR}/day05", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")]  
    cpu = IntCodeComputer(ops)
    out = cpu.run(5)
    print(out)   

773660


# Day 6


In [ ]:
class Satellite:
    def __init__(self, name, orbits=None):
        self.name = name
        self.orbits = orbits
        self.orbit_by = []
        self._total_orbits = None
        if orbits:
            orbits.add_satellite(self)

    def num_satellites(self):
        return len(self.orbits)
    
    def add_satellite(self, satellite):
        self.orbit_by.append(satellite)

    def set_orbits(self, orbits):
        self.orbits = orbits
        orbits.add_satellite(self)

    def orbits_path(self):
        o = self
        path = []
        while o.orbits is not None:
            o = o.orbits
            path.append(o)
        return path

    def total_orbits(self):
        if self._total_orbits is not None:
            return self._total_orbits

        self._total_orbits = len(self.orbits_path())
        return self._total_orbits

    def __repr__(self):
        return self.name

orbits = {}
with open(f"{INPUTS_DIR}/day06", "r") as fin:
    for line in fin:
        obj_name, sat_name = line.strip().split(")")
        obj = orbits.get(obj_name)
        if not obj:
            obj = Satellite(obj_name)
            orbits[obj_name] = obj

        sat = orbits.get(sat_name)
        if not sat:
            orbits[sat_name] = Satellite(sat_name, obj)
        else:
            sat.set_orbits(obj)

In [ ]:
sum([o.total_orbits() for o in orbits.values()])

247089

In [ ]:
san_path = orbits['SAN'].orbits_path()
san_path_set = set(san_path)
common = None
you_path = orbits['YOU'].orbits_path()
for sat in you_path:
    if sat in san_path_set:
        common = sat
        break

you_path.index(common) + san_path.index(common)

442

# Day 7

In [ ]:
with open(f"{INPUTS_DIR}/day07", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")] 

max_val = -1
for a,b,c,d,e in permutations([0,1,2,3,4],5):
    out = IntCodeComputer(ops.copy()).run([a, 0])
    out = IntCodeComputer(ops.copy()).run([b, out])
    out = IntCodeComputer(ops.copy()).run([c, out])
    out = IntCodeComputer(ops.copy()).run([d, out])
    out = IntCodeComputer(ops.copy()).run([e, out])
    if out > max_val:
        max_val = out

max_val

87138

In [ ]:
with open(f"{INPUTS_DIR}/day07", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")] 

def engine_feedback_loop(engines):
    signal = 0
    for engine in cycle(engines):
        if engine.halted:
            break

        signal = engine.run(signal)

    return signal

max_val = -1

for a,b,c,d,e in permutations([5,6,7,8,9],5):
    engines = [
        IntCodeComputer(ops.copy(), phase=a),
        IntCodeComputer(ops.copy(), phase=b),
        IntCodeComputer(ops.copy(), phase=c),
        IntCodeComputer(ops.copy(), phase=d),
        IntCodeComputer(ops.copy(), phase=e)     
    ]

    out = engine_feedback_loop(engines)
    if out > max_val:
        max_val = out

max_val

17279674

# Day 8

In [ ]:
with open(f"{INPUTS_DIR}/day08", "r") as fin:
    line = fin.read().strip()
    n = 25*6
    layers = [line[i:i+n] for i in range(0, len(line), n)]
    layer = min(layers, key=lambda x: x.count("0"))
    print(layer.count("1") * layer.count("2"))

1360


In [ ]:
with open(f"{INPUTS_DIR}/day08", "r") as fin:
    line = fin.read().strip()
    n = 25*6
    layers = [line[i:i+n] for i in range(0, len(line), n)]
    image = []
    for i in range(0, n):
        layer = 0
        while layers[layer][i] == "2":
            layer += 1
        image.append(layers[layer][i])

    for i, b in enumerate(image):
        if i % 25 == 0:
            print('')

        print("\u2B1B" if b == "0" else "\u2B1C", end='')



⬜⬜⬜⬜⬛⬜⬜⬜⬛⬛⬜⬛⬛⬜⬛⬛⬜⬜⬛⬛⬜⬜⬜⬛⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛
⬜⬜⬜⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛
⬜⬛⬛⬛⬛⬜⬜⬜⬛⬛⬜⬛⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜⬛⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬜⬛⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬜⬜⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛

# Day 9

In [ ]:
with open(f"{INPUTS_DIR}/day09", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")] 

comp = IntCodeComputer(ops)
comp.run(1)


4006117640

In [ ]:
with open(f"{INPUTS_DIR}/day09", "r") as fin:
    ops = [int(c) for c in fin.read().split(",")] 

comp = IntCodeComputer(ops)
comp.run(2)

88231

# Day 10

In [ ]:
satellites = []
line_of_sight = {}
with open(f"{INPUTS_DIR}/day10", "r") as fin:
    for i, line in enumerate(fin):
        for j, cell in enumerate(line): 
            if cell == "#":
                satellites.append((i,j))
                line_of_sight[(i,j)] = set()

max_los = 0
sat_max_los = None
for sat in satellites:
    for other in satellites:
        if sat == other:
            continue
        x = other[1] - sat[1]
        y = other[0] - sat[0]
        theta = math.atan2(y,x)
        line_of_sight[sat].add(theta)

    if len(line_of_sight[sat]) > max_los:
        max_los = len(line_of_sight[sat])
        sat_max_los = sat

max_los

284

# Day 11